# **Running Monte Carlo simulations in Python**


In [ ]:
# import the libraries we need
# these are all available by default in google colab, so we don't need to install them ourselves
import math
import copy
import random
import numpy as np
import matplotlib.pyplot as plt

First, we'll look at the harmonic oscillator as an example. We can write the potential energy of an oscillator (i.e., spring) as:
\begin{equation}
U(x) = \frac{1}{2}kx^2
\end{equation}

Here, k is the spring constant and x is the displacement or 'stretching' of the spring. The displacement can also be written as the difference between the current position and the equilibrium position:

\begin{equation}
x = r - r_{eq}
\end{equation}

We'll use Metropolis MC to look at the average energy, position, and displacement of a harmonic oscillator. This oscillation potential is frequently used to define bond vibrations in a lot of molecular simulations.

In [ ]:
### Example: HCl molecule ###

# defining some constants
kb = 1.380649e-23 #Boltzmann constant, J / K
k = 480 #spring constant, N/m
r_eq = 1.27e-10 #equilibrium bond length, m

def harmonic(r):
  """ Defining our potential energy from the above equation """
  pot = 0.5 * k* (r - r_eq)**2
  return pot

Probability of acceptance in the Boltzmann distribution:

\begin{equation}
P_a = \exp\left(-\frac{E_f - E_i}{k_B T}\right)
\end{equation}

Where $E_f$ and $E_i$ are the energies of the new and original states respectively.

In [ ]:
# Running the Monte Carlo sim

acc = 0 #counter of accepted moves
dst = 0.0 #distances
ener = 0.0 #energy

T = 298 #temperature in Kelvin
r = 1e-10 #some initial starting distance
points = 1000 #number of points to run the simulation for
step = 1e-11

for points in range(1, points):
  r_new = r + step * ( random.random() - 0.5) * 2.0
  e     = harmonic(r)
  e_new = harmonic(r_new)
  if e_new < e:	# lower E move always accepted
    r = r_new
    e = e_new
    acc = acc + 1
  else:	# higher E move accepted depending on probability
    A_mov = math.exp(-(e_new - e)/(kb*T) )
    if A_mov > random.random():
        r = r_new
        e = e_new
        acc = acc + 1
				# Update regardless of acceptance!
  dst  = dst + r
  ener = ener + e

In [ ]:
# Print out the averages
dst_av = dst / float(points)
ener_av = ener / float(points)

print('Acceptance is:', 100 * acc / float(points), '%')
print(' ')
print('Monte Carlo Averages:')
print('Mean distance, nm:                 ', dst_av * 1e9)
print('Mean MC potential energy, J: ', ener_av)

Acceptance is: 45.945945945945944 %
 
Monte Carlo Averages:
Mean distance, nm:                  0.12684676896854818
Mean MC potential energy, J:  2.1903084883259437e-21


Now let's set up an MC system to simulate a 2D 'box' of Argon molecules using the Lennard-Jones potential:



In [ ]:
# Set the number of atoms in the box
n_atoms = 25

# Set the number of Monte Carlo moves to perform
n_moves = 1000

# Set the size of the box (in nm)
box_size = [ 1.5, 1.5 ]

# The maximum amount that the atom can be translated by
max_translate = 0.05    # nm

# Simulation temperature
temperature = 298   # kelvin

# Give the Lennard Jones parameters for the atoms
# in our case, let's look at Argon
sigma = 0.34        # nm
epsilon = 120*kb       # J

In [ ]:
# Create an array to hold the coordinates of the atoms
coords = []

# Randomly generate the coordinates of the atoms in the box
for i in range(0,n_atoms):
    # Note "random.uniform(x,y)" would generate a random number
    # between x and y
    coords.append( [random.uniform(0,box_size[0]), random.uniform(0,box_size[1])] )

Since we're working with code from scratch right now, we have to write a handful of helper functions to handle simulation things that a software would otherwise do for us.

In [ ]:
def make_periodic(x, box):
    """Function to apply periodic boundaries"""
    while (x < -0.5*box):
        x += box
    while (x > 0.5*box):
        x -= box
    return x

def wrap_into_box(x, box):
    """Function to wrap the coordinates into a box"""
    while (x > box):
        x -= box
    while (x < 0):
        x += box
    return x

def calculate_energy():
    """Calculate the energy of atoms"""

    # Loop over all pairs of atoms and calculate
    # the LJ energy
    total_energy = 0

    for i in range(0,n_atoms-1):
        for j in range(i+1, n_atoms):
            delta_x = coords[j][0] - coords[i][0]
            delta_y = coords[j][1] - coords[i][1]

            # Apply periodic boundaries
            delta_x = make_periodic(delta_x, box_size[0])
            delta_y = make_periodic(delta_y, box_size[1])

            # Calculate the distance between the atoms
            r = math.sqrt( (delta_x*delta_x) + (delta_y*delta_y) )

            # E_LJ = 4*epsilon[ (sigma/r)^12 - (sigma/r)^6 ]
            e_lj = 4.0 * epsilon * ( (sigma/r)**12 - (sigma/r)**6 )

            total_energy += e_lj

    # return the total energy of the atoms
    return total_energy

# We need to make some functions for periodic boundary conditions, just like in MD

In [ ]:
#counters of accepted and rejected coordinates
acc = 0
rej = 0

#perform the simulation!
for move in range (1, n_moves + 1):
  old_e = calculate_energy() # calculate the old energy
  old_coords = copy.deepcopy(coords) # save the old coordinates

  atom = random.randint(0, n_atoms-1) # pick a random atom to move

  # translate by a random amount in each direction
  del_x = random.uniform(-max_translate, max_translate)
  del_y = random.uniform(-max_translate, max_translate)
  coords[atom][0] += del_x
  coords[atom][1] += del_y

  # wrap coordinates back into the box. this basically enforces periodic conditions
  coords[atom][0] = wrap_into_box(coords[atom][0], box_size[0])
  coords[atom][1] = wrap_into_box(coords[atom][1], box_size[1])

  new_e = calculate_energy() # calculate the new energy

  accept = False # initializing the Boolean variable
  if (new_e <= old_e):
    accept = True # lower E move always accepted
  else:
    x = math.exp(-(new_e - old_e)/(kb*T))
    if (x >= random.uniform(0.0, 1.0)): # higher E move accepted depending on probability
      accept = True
    else:
      accept = False

  if accept:
    acc += 1
    total_e = new_e
  else:
    rej += 1
    coords = copy.deepcopy(old_coords) #if rejected, we need to go back to the original config
    total_e = old_e

  if move % 10 == 0:
    print("%s %s %s %s" % (move, total_e, acc, rej)) #print otu info every 10 moves

10 1.2992279894547096e-18 1 9
20 1.2751853898172142e-18 4 16
30 1.2558317759693366e-18 6 24
40 1.1756771409144647e-18 9 31
50 1.1714686713872297e-18 12 38
60 1.1570159084185288e-18 15 45
70 1.1569562681690683e-18 17 53
80 1.1586337600486235e-18 19 61
90 1.15565024842821e-18 20 70
100 1.15565024842821e-18 20 80
110 1.1497003172381465e-18 22 88
120 1.1497003172381465e-18 22 98
130 1.1441569729495885e-18 23 107
140 1.1441569729495885e-18 23 117
150 1.1522028254422752e-18 25 125
160 1.1516779486757255e-18 26 134
170 1.1516779486757255e-18 26 144
180 1.1516779486757255e-18 26 154
190 1.1664552649646091e-18 27 163
200 1.1671783083602372e-18 28 172
210 1.1444661941475816e-18 30 180
220 1.1444661941475816e-18 30 190
230 1.1444661941475816e-18 30 200
240 1.1408868743258622e-18 32 208
250 1.1409525395297892e-18 33 217
260 1.1409525395297892e-18 33 227
270 1.1284812915667756e-18 35 235
280 1.1284812915667756e-18 35 245
290 1.1104494699474413e-18 37 253
300 1.070323897970556e-18 39 261
310 1.05532

# Exercise 1: The Ising model

The Ising model is a simple model of ferromagnetism. Imagine a magnet is made up of dipoles (electron spins) that can point up (+1) or down (-1). In real life, they interact with each other to give rise to magnetic behaviour. The energy of the system can be defined as such:

\begin{equation}
E = -J\sum_{\left<ij\right>} s_i s_j
\end{equation}

$J$ is the interaction strength between the spins, which you can set to 4. The net **magnetization** is the sum of all of the spin values.
This is what the 2D Ising model looks like. (The 1D example would just be one row of this.)
<center>
<img src='https://drive.google.com/uc?id=1lHBi0ZymQQo9_FdwfZ1O8jD3DnsIwLPh'
width ="200"
height="200"/>
</center>

Try to write code to run a 1D simulation of the Ising model. These are the steps you'll need to work through:

*   Create array of dipoles, initial state: randomize all the spins.
*   Calculate energy & magnetization of the state
*   Implement the Metropolis algorithm:
      *  create new state: flip 1 spin randomly
      *  calculate new total energy
      *  calculate acceptance probability
      *  decide whether to accept or reject new state
      *  store 'new' energy & magnetization
      *  repeat

If you're familiar with Python and want an extra challenge, try the 2D problem as well!

In [ ]:
# 1D Ising Model

N = 10
steps = 100

# create an array of random integers of either 0 or 1
spins = np.random.randint(2, size=N)

# change the 0s in the array to -1
for idx, i in enumerate(spins):
    if i == 0:
        spins[idx] = -1

print(spins)
def energy(arr):
    e=0
    # calculating energy of the state
    for j in range(4):
        e += spins[j]*spins[j+1]
    return -4 * e

m_vals = []

m = sum(spins)/N
e = energy(spins)

for k in range(steps):

    flip = np.random.randint(N) # choose the spin to be flipped

    spins[flip] *= -1 # flipping the chosen spin

    ne = energy(spins)

    if ne <= e:
        m = sum(spins)/N
    elif ne > e:
        if random.random() < np.exp(-(ne - e)):
            m = sum(spins)/N
        else:
            spins[flip] *= -1 # flip the spin back if we are not accepting this state
            m = sum(spins)/N

    e = energy(spins)

    # adding magnetization to list
    m_vals.append(m)



[-1 -1 -1 -1  1  1 -1  1 -1 -1]


In [ ]:
#2D

N = 5
steps = 50000

spins = np.random.randint(2, size=(N, N))

for i in range(N):
    for j in range(N):
        if spins[i, j] == 0:
            spins[i, j] = -1
print(spins)

def energy_2D(arr):
    '''This function calculates the energy of a spin state'''

    E = 0
    e = 0

    # calculating the 'vertical' interactions
    for i in range(N-1):
        for j in range(N):
            e += arr[i+1, j]*arr[i, j]
            E += e
    # calculating the 'horizontal' interactions
    for i in range(N):
        for j in range(N-1):
            e = arr[i, j+1]*arr[i, j]
            E += e

    return -1 * E


def mag(arr):
    return sum(map(sum, arr))

m_vals = []

e = energy_2D(spins)
m = mag(spins)

m_vals.append(m)

for k in range(steps):

    flip_i = np.random.randint(N)
    flip_j = np.random.randint(N)

    spins[flip_i, flip_j] *= -1 # flipping the chosen spin

    ne = energy_2D(spins) # calculate the new energy

    if ne > e:
        if random.random() < np.exp(-(ne - e)):
            e = ne
            m = mag(spins)
        else:
            spins[flip_i, flip_j] *= -1
            e = energy_2D(spins)
    else:
        e = ne
        m = mag(spins)


    # adding magnetization to list
    m_vals.append(m)

[[ 1  1  1  1 -1]
 [-1  1  1  1 -1]
 [-1  1 -1 -1  1]
 [-1  1  1 -1 -1]
 [ 1 -1  1 -1  1]]
